# download

> This module downloads the raw data from CDS and saves it in the local directory


In [ ]:
#| default_exp download


In [ ]:
#| hide
from nbdev.showdoc import *

We use a similar approach to the one in the tutorial to download the data
to local storage.

In [ ]:
#| export
import os
import hydra
import cdsapi
from core import _expand_path
from omegaconf import DictConfig, OmegaConf

In [ ]:
#| exporti
def _validate_query(
        query_body: DictConfig
    )->bool:
    '''
    Check that the query is valid
    '''

    required_keys = ['product_type', 'variable', 'year', 'month', 'day', 'time', 'area', 'data_format', 'download_format']
    if not all([key in query_body.keys() for key in required_keys]):
        print(f"Missing required key in query. Required keys are {required_keys}")
        print("Query validation failed")
        raise ValueError("Invalid query")
        
    else:
        return query_body

In [ ]:
#| export
def fetch_MDG_GADM(
        output_file: str="gadm41_MDG.gpkg" # file path to save the GADM data
    )-> str:
    '''
    Fetch the GADM data for Madagascar
    https://geodata.ucdavis.edu/gadm/gadm4.1/gpkg/gadm41_MDG.gpkg
    '''

    output_file_path = _expand_path(output_file)
    if os.path.exists(output_file_path):
        print("GADM data already exists")
        return output_file_path
    
    print("Fetching GADM bounding box data for Madagascar")
    os.system("curl --output {} https://geodata.ucdavis.edu/gadm/gadm4.1/gpkg/gadm41_MDG.gpkg".format(output_file))
    print("GADM data fetched")
    
    return output_file_path

In [ ]:
#| export
def download(
        cfg: DictConfig,  # hydra configuration file
        output_dir: str = "data/input/", # output directory
        dataset: str = "reanalysis-era5-pressure-levels", # dataset to download
        testing: bool = True  # testing mode
    )->None:
    '''
    Send the query to the API and download the data
    '''
    
    client = cdsapi.Client()
    
    query = _validate_query(cfg.query)
    
    # Send the query to the client
    if not testing:
        client.retrieve(dataset, query).download(os.path.join(_expand_path(output_dir), "{}_{}.nc".format(query.year, query.month)))
    else:
        print(f"Testing mode. Not downloading data. Query is {query}")

    print("Done")

In [ ]:
#| export
@hydra.main(config_path="../conf", config_name="config", version_base=None)
def main(cfg: DictConfig) -> None:
    download(cfg=cfg)

In [ ]:
#| export
try: from nbdev.imports import IN_NOTEBOOK
except: IN_NOTEBOOK=False

if __name__ == "__main__" and not IN_NOTEBOOK:
    print('Running from __main__ ...')

    main()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()